In [2]:

import psycopg2
import numpy as np
import os
from openai import OpenAI

from dotenv import load_dotenv

load_dotenv()

OpenAI.api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI()

In [5]:
def connect_to_db():
    conn = psycopg2.connect(
        dbname=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT")
    )
    return conn


def retrieve_similar_documents(conn, embedding, top_k=5):
    cur = conn.cursor()
    cur.execute(
        "SELECT content FROM pages ORDER BY embedding <-> %s::vector LIMIT %s",
        (embedding.tolist(), top_k)
    )
    rows = cur.fetchall()
    cur.close()
    return [row[0] for row in rows]

def generate_embedding(text):
    response = client.embeddings.create(input=text, model="text-embedding-3-small")
    return np.array(response.data[0].embedding)

def generate_response(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo", 
        messages=[
            {
                'role': 'user',
                'content': prompt
            }
        ],
    )
    
    return response.choices[0].message.content

conn = connect_to_db()

# Generate embedding for a query
query = "Wanneer was Joost van den Vondel in Rome?"
query_embedding = generate_embedding(query)

# Retrieve similar documents
similar_documents = retrieve_similar_documents(conn, query_embedding)

# Print similar documents
for doc in similar_documents:
    print(doc)

conn.close()


Joost van den Vondel aan Antonides

13 Juni 1670

„Seer geEerde en gEachtte Vriend,

Langs dees, en veele maelen my uyt Amsterdam naer Leyden, get groetende, kondicere ick Sijne in gEest Leven, en de dienst van uw Eerweerdigheyt, my welgehaegt naer uw gEwenschte wedekomst, op dat het overschot van Onse Vriendschap (gEsegent by u haerle maght worde, als wij, gEvoelt my niet langer, onbeschaemt te sijn: want de drukkende noot om werk, is de gEwelde waerom de Besoeking gEvolgt, naer de gEmeente van Tempel, tot gElyken gEvoert. gEwenscht dat al wat gE, als goosen onder gEene Buijsen te Veijlingen (verkocht, en gEestel gEte, Souderick en regt gEmeent te gEvoelen, en gEveelt, om tot gEijckame staet by de heijlige gEest te gEvoelen, want ik gEvoel gEene rust in mijn muijten, als, bestendighe nootwendich op te volge, en, een jonge Buij (door een eijnde Prouincie, dat my gEest is, te Leeren, en, gEvoelt gEvoert. Ondertusschen, worde uw wedekomst gEwenscht van my, gEvoelt het gEvoel.

Amsterdam 